In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns  

In [ ]:
from sklearn.datasets import load_breast_cancer
data = load_breast_cancer()


In [ ]:
data.keys()

In [ ]:
data['feature_names'][:10]

In [ ]:
data['target_names']

In [ ]:
selected_features = data['feature_names'][:10]
data_df = pd.DataFrame(data['data'], columns = data['feature_names'])[selected_features]
data_df['target'] = data['target']
data_df = data_df.sample(frac = 1)

In [ ]:
data_df

In [ ]:
x_train = data_df.iloc[:400][selected_features]
y_train = data_df.iloc[:400]['target']
x_test = data_df.iloc[400:][selected_features]
y_test = data_df.iloc[400:]['target']

In [ ]:
from sklearn import svm

clf = svm.SVC(kernel='linear', C=1,probability=True)
clf_fit = clf.fit(x_train, y_train)
y_pred= clf_fit.predict(x_test)

In [ ]:
from sklearn.metrics import accuracy_score

# accuracy: (tp + tn) / (p + n)
accuracy = accuracy_score(y_test, y_pred)
print('Accuracy: %f' % accuracy)

In [ ]:
from sklearn.metrics import plot_roc_curve
from sklearn.exceptions import NotFittedError
import matplotlib.pyplot as plt
%matplotlib inline
disp = plot_roc_curve(clf_fit,x_test,y_test)

# LIME SVM

In [ ]:
import lime
import lime.lime_tabular


In [ ]:
explainer = lime.lime_tabular.LimeTabularExplainer(np.array(x_train),
                    feature_names = list(data['feature_names'][:10]),
                    class_names=['malignant', 'benign'],                            
                    verbose=True, mode='classification')


In [ ]:
exp = explainer.explain_instance(x_test.iloc[0], clf_fit.predict_proba, num_features=10)
exp.show_in_notebook(show_table=True, show_all=False)
exp.save_to_file('svm_lime_1.html')

In [ ]:
exp = explainer.explain_instance(x_test.iloc[80], clf_fit.predict_proba, num_features=10)
exp.show_in_notebook(show_table=True, show_all=False)
exp.save_to_file('svm_lime_2.html')

# SHAP SVM

In [ ]:
import shap

In [ ]:
svm_explainer = shap.KernelExplainer(clf_fit.predict,x_test)
svm_shap_values = svm_explainer.shap_values(x_test)

In [ ]:
shap.summary_plot(svm_shap_values, x_test)